# Exercise: Features and supervised learning

In this exercise we look at different methods for extracting features from images and using them for supervised learning. We well concentrate mainly on example images from textures and try to discriminate between different textures.

In the first exercise we will lokk at binary local patterns and in the second exercise we will gather features from the glcm and train a classifier.


In [ ]:
import numpy as np
import skimage
import skimage.data
import skimage.io
import skimage.future
import skimage.segmentation
import skimage.feature
import sklearn.cluster
import cv2

# for displaying images in jupyter
from matplotlib import pyplot as plt

%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 10]
#mpl.rcParams['figure.dpi']= 400

## Exercise 1: Texture Recognition using Binary Local Patterns and Histograms

In this exercise we will look at texture images from two different classes and extract the binary local patterns.

We first load two images from the *describable texture database (dtd)*.

In [ ]:
im_1 = skimage.io.imread('dtd/images/crosshatched/crosshatched_0044.jpg')
im_2 = skimage.io.imread('dtd/images/waffled/waffled_0029.jpg')

plt.subplot(2, 1, 1)
plt.imshow(im_1)
plt.subplot(2, 1, 2)
plt.imshow(im_2)

### Use grayscale images

We will only use the gray images, so we will convert them to grayscale. By default, this will turn them into float64 images if we use the ```skimage.color.rgb2gray``` function. As we want to continue to work with 8bit images, we convert them back to ubyte using ```skimage.img_as_ubyte```:

In [ ]:
im_1_gray = skimage.img_as_ubyte(skimage.color.rgb2gray(im_1))
plt.imshow(im_1_gray, cmap='gray')

### Exercise 1.1. Convert to grayscale

Do the same for the other image.

In [ ]:
# Solution
im_2_gray = skimage.img_as_ubyte(skimage.color.rgb2gray(im_2))
plt.imshow(im_2_gray, cmap='gray')

### Exercise 1.2 Calculate the local binary patterns (LBP)

Calculate the local binary patterns using the function ```skimage.feature.local_binary_pattern```. Check the description of the function for different variants of the pattern calculation. 

Visualize the patterns as image. What is the range of the resulting data? Do you think we could we use the values directly to distinquish between the two textures?

In [ ]:
# Solution
lbp_1_1 = skimage.feature.local_binary_pattern(im_1_gray, P=8, R=2.0, method='default')
#feature_1 = np.stack([lbp_1_1, lbp_1_2], axis=2)
print(lbp_1_1.shape)
print(np.max(lbp_1_1))
print(lbp_1_1.dtype)


lbp_2_1 = skimage.feature.local_binary_pattern(im_2_gray, P=8, R=2.0, method='ror')

plt.subplot(2, 1, 1)
plt.imshow(lbp_1_1)
plt.subplot(2, 1, 2)
plt.imshow(lbp_2_1)

### Exercise 1.3: Texture Segmentation with LBP

In order to use the LBP for (texture) segmentation, we can not use the pixel values directly. So an often used approach is to use the histogram of the LBP as feature. For texture segmentation we want to calculate this histogram on a subwindow.


#### Sliding window approach

For a sliding window approach, skimage provides two utility function to calculate blocks from an image. One is ```skimage.util.view_as_blocks``` that divides the image into non-overlapping blocks and the other is ```skimage.util.view_as_windows``` that divides the image into overlapping windows. For the latter approach, we have to be careful to not generate too many windows, so in the sliding window approach we often use some step value greater than 1.

Use the function ```skimage.util.view_as_blocks``` and see how to access the blocks. Plot the blocks as images.



In [ ]:
# Solution

blocks_1 = skimage.util.shape.view_as_blocks(im_1_gray, block_shape=(80,80))
print(blocks_1.shape)

index = 1
for i in range(blocks_1.shape[0]):
    for j in range(blocks_1.shape[1]):
        plt.subplot(blocks_1.shape[0], blocks_1.shape[1], index)
        plt.imshow(blocks_1[i,j], cmap='gray')
        index += 1

Try the same with the function ```skimage.util.view_as_windows```. What could be the advantage of this function over the other one?

In [ ]:
# Solution
blocks_1 = skimage.util.shape.view_as_windows(im_1_gray, window_shape=(80,80), step=(40,40))
print(blocks_1.shape)

index = 1
for i in range(blocks_1.shape[0]):
    for j in range(blocks_1.shape[1]):
        plt.subplot(blocks_1.shape[0], blocks_1.shape[1], index)
        plt.imshow(blocks_1[(i,j)], cmap='gray')
        index += 1

#### Histogram calculation
Calculate the histogram (```np.histogram```) of the LBP in 2 subwindows of the first image and then in two subwindows of the second texture image and plot all four histograms. Use 255 bins for the histogram. ```np.histogram``` returns both the histogram values and the edges of the bins, there is one more edge than number of bins)

Plot the resulting histograms.


In [ ]:
# Solution
blocks_lbp_image_1 = skimage.util.shape.view_as_windows(lbp_1_1, window_shape=(80,80), step=80)
blocks_lbp_image_2 = skimage.util.shape.view_as_windows(lbp_2_1, window_shape=(80,80), step=80)

hist_1_1, bins = np.histogram(blocks_lbp_image_1[0,0], bins=255)
hist_1_2, _ = np.histogram(blocks_lbp_image_1[1,0], bins=255)
hist_2_1, _ = np.histogram(blocks_lbp_image_2[0,0], bins=255)
hist_2_2, _ = np.histogram(blocks_lbp_image_2[1,0], bins=255)

In [ ]:
# Solution
plt.subplot(2, 2, 1)
#ax.bar(edges[:-1], frq, width=np.diff(edges), ec="k", align="edge")
plt.bar(bins[:-1], hist_1_1)
plt.subplot(2, 2, 2)
hist_1_2[0].shape
plt.bar(bins[:-1], hist_1_2)
plt.subplot(2, 2, 3)
hist_1_2[0].shape
plt.bar(bins[:-1], hist_2_1)
plt.subplot(2, 2, 4)
hist_1_2[0].shape
plt.bar(bins[:-1], hist_2_2)

#### Histogram comparison

There are different functions to compare histograms. One possibility is ```scipy.stats.chisquare``` for the so called chi square difference. Other implementation of histogram comparisons are available in OpenCV (https://docs.opencv.org/3.4/d8/dc8/tutorial_histogram_comparison.html).

If you want to use the ```scipy.stats.chisquare```, it will fail if any of the histogram values are 0. This can be solved by adding 1 to the histogram.

Calculate the histogram differences between histograms from the same image and the histogram from different images. 

Are the results as expected?


In [ ]:
import scipy.stats
print(scipy.stats.chisquare(hist_1_1+1, hist_1_1+1))
print(scipy.stats.chisquare(hist_1_1+1, hist_1_2+1))
print(scipy.stats.chisquare(hist_2_1+1, hist_2_2+1))
print(scipy.stats.chisquare(hist_1_1+1, hist_2_1+1))
print(scipy.stats.chisquare(hist_1_2+1, hist_2_2+1))

## Exercise 2: Semantic Segmentation using statistical features, GLCM features and supervised learning

In the next exercise we want to use a feature vector from statistical data and use it for supervised learning. For that we will again use a sliding window approach, so first we divide the images again into sliding window views. Use few windows (i.e. a large step size) first and increase the number of windows when all is implemented. Try a block size for example of 40 by 40 pixels.

In [ ]:
# Solution
blocks_1 = skimage.util.shape.view_as_windows(im_1_gray, window_shape=40, step=20)
blocks_1_flat = blocks_1.reshape(-1, blocks_1.shape[2], blocks_1.shape[3])

### Exercise 2.1: Statistical features

Calculate the mean value (```np.mean```) and the standard deviation (```np.std```) of a block. This will be our first features.

In [ ]:
# Solution
# the `max` or the `median` value of each blocks.
mean = np.mean(blocks_1_flat[0])
std = np.std(blocks_1_flat[0])
print(mean)
print(std)

### Exercise 2.2. GLCM

Next we want do calculate the GLCM (using skimage.feature.greycomatrix). The matrix is a square nxn matrix, where n is the number of levels. For uint8 images n would be 256, however this is probably too large, we we use only 16 levels which can be achieved by dividing the image by 16 (usind the // operator in python for integer division).

```image = blocks_1_flat[0] // 16```

Calculate a matrix and indicate the correct number of levels. Check the documentation about the arguments.

In [ ]:
#Solution
image = blocks_1_flat[0] // 16
glcm = skimage.feature.greycomatrix(image, distances=[1], angles=[0], levels=16, symmetric=True, normed=False)

#### Multiple GLCMs
It is possible to calculate different GLCMs for different pixel neighborhoods and different orientations. Change the arguments to calculate GLCMs for the distances 1 and 2 pixels and for horizontal and vertical orientations.

In [ ]:
# Solution
glcm = skimage.feature.greycomatrix(image, distances=[1,2], angles=[0, np.pi / 2.0], levels=16, symmetric=True, normed=False)


### Exercise 2.3 Features
#### Properties from GLCMs

```skimage.feature.greycoprops``` allows to calculate different features from the GLCM such as contrast and dissimilarity. The full list of supported features is

```['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation', 'ASM']```

Calculate those features and print them.

In [ ]:
# Solution
for prop in ['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation', 'ASM']:
    print(skimage.feature.greycoprops(glcm, prop=prop))

#### Feature calculation
Define a function that takes an input image and calculates a feature vector from it.

In [ ]:
# Solution (part)
def calc_features(image: np.ndarray):
    features = np.asarray([np.mean(image), np.std(image)])
    glcm = skimage.feature.greycomatrix(image, [1,2], [0, np.pi / 2], levels=16, symmetric=True, normed=False)
    for prop in ['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation', 'ASM']:
        features = np.concatenate((features, skimage.feature.greycoprops(glcm, prop=prop).flatten()))
    return features
f = calc_features(blocks_1_flat[0] // 16)
print(f.shape)


Calculate the features for every window in the image and put them together to a large vector of features.


In [ ]:
# Solution
f = [calc_features(image // 16) for image in blocks_1_flat]
data = np.stack(f, axis=0)


#### Complete the data sets for training
For supervised learning, we need a label for each feature, lets assign the first image the label 0 and the second image the label 1. Put together a label vector for the data set 1 and then put together the features and labels for the data set 2. Finally concatenate both data sets and labels together.

In [ ]:
# Solution
# generate dataset suitable for the supervised learning methods in sklearn
blocks_1 = skimage.util.shape.view_as_windows(im_1_gray, window_shape=40, step=20)
blocks_1_flat = blocks_1.reshape(-1, blocks_1.shape[2], blocks_1.shape[3])
f_1 = [calc_features(image // 16) for image in blocks_1_flat]
data_1 = np.stack(f_1, axis=0)
label_1 = np.empty(data_1.shape[0])
label_1.fill(0)

blocks_2 = skimage.util.shape.view_as_windows(im_2_gray, window_shape=40, step=20)
blocks_2_flat = blocks_2.reshape(-1, blocks_2.shape[2], blocks_2.shape[3])
f_2 = [calc_features(image // 16) for image in blocks_2_flat]
data_2 = np.stack(f_2, axis=0)
label_2 = np.empty(data_2.shape[0])
label_2.fill(1)

# create a full data set
data = np.concatenate([data_1, data_2])
label = np.concatenate([label_1, label_2])




In [ ]:
print(data.shape, label.shape)

### 2.4: Supervised learning

In order to use any of the available supervised learning approaches, we *should* normalize the data (read *must* :-)). There is a function ```sklearn.preprocessing.normalize``` that can do that, alternatively we could normalize the features to be in the range between 0 and 1 when we put then together. 

When using ```sklearn.preprocessing.normalize```, we do not want to normalize individual feature vector (axis=1, the default), but along all values of a feature (axis = 0). So the call should be

```data_norm = sklearn.preprocessing.normalize(data, axis=0)```

Also our data is not shuffled randomly, which might impact the result for some approaches. We can shuffle a data set and the label together using 

```sklearn.utils.shuffle(data, label)```.


In [ ]:
# Solution
data_norm = sklearn.preprocessing.normalize(data, axis=0)
# shuffle the data sets
data_s, label_s = sklearn.utils.shuffle(data_norm, label)

#### Classifiers

We are finally ready to train a classifier on the data. We will use a decision tree and a support vector machine. The decision tree is implemented in 

```dtc = sklearn.tree.DecisionTreeClassifier()```

which will return a classifier object that has the member function fit

```dtc.fit(data_s, label_s)```

There a different variants of support vector machines in the package sklearn.svm. For our problem, we can use the LinearSVC:

```svm = sklearn.svm.LinearSVC()```

Train both classifiers and predict the (training) data using 

```dtc.predict(data_s)```.

How good is the result? How do we evaluate the result?

In [ ]:
#Solution
# use a decision tree classifier
dtc = sklearn.tree.DecisionTreeClassifier()
dtc.fit(data_s, label_s)
# prediction
labels_p = dtc.predict(data_s)
sklearn.metrics.confusion_matrix(label_s, labels_p)

In [ ]:
# Solution
svm = sklearn.svm.LinearSVC()
svm.fit(data_s, label_s)
labels_p = svm.predict(data_s)
sklearn.metrics.confusion_matrix(label_s, labels_p)


### Exercise 3 (optional): Further evaluation

- The results are probably quite good. How can we look at some of the data to understand the results better?
- Try loading two another images from the same textures and run the classification. Does it still work?

In [ ]:
plt.scatter(data_s[:,0], data_s[:,3], c=label_s)